# BERT Test Implementation 

@authors: taiyyoson, ebeckk

In [8]:
%pip install transformers datasets scikit-learn torch
%pip install textstat

Note: you may need to restart the kernel to use updated packages.
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.4 MB/s eta 0:00:00a 0:00:01
Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import textstat

ModuleNotFoundError: No module named 'nltk'

In [9]:
nltk.download('punkt_tab')

NameError: name 'nltk' is not defined

In [3]:
## sample test data to run BERT on
data = {
    "text": ["I love this movie!", "This is terrible.", "Amazing experience!", "Worst day ever.", "So good!", "Really bad."],
    "label": [1, 0, 1, 0, 1, 0]  # 1 = Positive, 0 = Negative
}

df = pd.DataFrame(data)

## split data
train_X, test_X_val, train_y, test_y = train_test_split(df["text"], df["label"], test_size=0.2)

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## tokenize the texts w tokenizer
train_encodings = tokenizer(list(train_X), truncation=True, padding=True)
val_encodings = tokenizer(list(test_X_val), truncation=True, padding=True)

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Build datasets
train_dataset = TextDataset(train_encodings, list(train_y))
val_dataset = TextDataset(val_encodings, list(test_y))

In [5]:
## load BERT model (pretrained)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

## setup training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    eval_strategy="epoch",     # evaluate every epoch
    save_strategy="epoch",           # save model every epoch
    logging_dir='./logs',            # directory for logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,0.643010
2,No log,0.614255
3,No log,0.600028


/opt/anaconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=3, training_loss=0.6548411051432291, metrics={'train_runtime': 57.4742, 'train_samples_per_second': 0.209, 'train_steps_per_second': 0.052, 'total_flos': 36999992160.0, 'train_loss': 0.6548411051432291, 'epoch': 3.0})

In [6]:
trainer.evaluate()

/opt/anaconda3/envs/ml/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.6000279188156128,
 'eval_runtime': 3.3835,
 'eval_samples_per_second': 0.591,
 'eval_steps_per_second': 0.296,
 'epoch': 3.0}

In [ ]:
## chatgpt predict method
'''def predict(texts):
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
    outputs = model(**encodings)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    return predictions

# Test it
sample_texts = ["I hated that.", "What a wonderful experience."]
print(predict(sample_texts))'''

RuntimeError: Placeholder storage has not been allocated on MPS device!